In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)


class Basnet(nn.Module):
    def __init__(self, num_classes=58):
        super().__init__()

        # ---- Stem ----
        self.stem = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        # ---- Residual Stages ----
        self.stage1 = nn.Sequential(
            ResidualBlock(64, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)      # 32 → 16
        )

        self.stage2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128),
            nn.MaxPool2d(2)      # 16 → 8
        )

        self.stage3 = nn.Sequential(
            ResidualBlock(128, 256),
            ResidualBlock(256, 256),
            nn.MaxPool2d(2)      # 8 → 4
        )

        self.stage4 = nn.Sequential(
            ResidualBlock(256, 512),
            nn.MaxPool2d(2)      # 4 → 2
        )

        # ---- Classifier ----
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.classifier(x)
        return x


In [ ]:
MODEL_PATH = "models/Basnet_V1.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Basnet = Basnet().to(device)
Basnet.load_state_dict(torch.load(MODEL_PATH, map_location=device))
Basnet.eval()
print("Model loaded successfully on device:", device)


Model loaded successfully on device: cpu


In [ ]:
from torchvision import transforms,datasets
from torch.utils.data import DataLoader

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])




test_dir = "Final_Test_data"
test_ds = datasets.ImageFolder(test_dir, transform=val_transform)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

history = {"test_loss": [], "test_acc": []}
test_loss = 0.0
correct = 0
total = 0
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = Basnet(images)
                    loss = criterion(outputs, labels)
                    test_loss += loss.item()

                    preds = outputs.argmax(1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)


avg_test_loss = test_loss / len(test_loader)
test_acc = correct / total

history["test_loss"].append(avg_test_loss)
history["test_acc"].append(test_acc)

#prining by comparing true labels from data to predicted labels
# with open("predicted_vs_true_labels.txt", "w") as f:
#     print("Predicted vs True Labels:")
#     with torch.no_grad():
#         for images, labels in test_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = Basnet(images)
#             preds = outputs.argmax(1)
#             f.write(f"Predicted: {preds.item()}, True: {labels.item()}\n")

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Predicted vs True Labels:
Test Loss: 0.8068, Test Accuracy: 0.8058
